## Setup imports

In [14]:
import sys
import os
import logging
import numpy as np
import torch
import monai.networks.nets as nets
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    CropForegroundd,
    ToTensord,
    RandFlipd,
    RandAffined,
    SpatialPadd,
    Activationsd,
    Resized,
)
from transforms import (
    CTWindowd,
    CTSegmentation,
    RelativeCropZd,
    RandGaussianNoised,
)
from monai.data import DataLoader, Dataset, PersistentDataset, CacheDataset
from torchsampler import ImbalancedDatasetSampler
from monai.transforms.croppad.batch import PadListDataCollate
from monai.utils import NumpyPadMode, set_determinism
from monai.utils.enums import Method
from monai.config import print_config
from sklearn.model_selection import train_test_split
from trainer import Trainer
from validator import Validator
from tester import Tester
from utils import (
    multi_slice_viewer,
    setup_directories,
    get_data_from_info,
    large_image_splitter,
    calculate_class_imbalance,
    create_device,
    balance_training_data,
    pickle_hashing,
)
from test_data_loader import TestDataset
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 0.5.dev2115
Numpy version: 1.20.2
Pytorch version: 1.8.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: c9990b040832792aa897b27a07180d4629cb8de3

Optional dependencies:
Pytorch Ignite version: 0.4.4
Nibabel version: 3.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 8.1.2
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.9.0+cu111
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.59.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup directories

In [15]:
dirs = setup_directories()

## Setup torch device

In [16]:
# pass "cuda" to use the GPU
device, using_gpu = create_device("cuda")

## Load and randomize images

In [17]:
# HACKATON image and segmentation data
hackathon_dir = os.path.join(dirs["data"], 'HACKATHON')
map_fn = lambda x: (x[0], int(x[1]))
with open(os.path.join(hackathon_dir, "train.txt"), 'r') as fp:
    train_info_hackathon = [map_fn(entry.strip().split(',')) for entry in fp.readlines()]
image_dir = os.path.join(hackathon_dir, 'images', 'train')
seg_dir = os.path.join(hackathon_dir, 'segmentations', 'train')
_train_data_hackathon = get_data_from_info(
    image_dir, seg_dir, train_info_hackathon, dual_output=False
)
_train_data_hackathon = large_image_splitter(_train_data_hackathon, dirs["cache"])
    
# PSUF data
"""psuf_dir = os.path.join(dirs["data"], 'psuf')
with open(os.path.join(psuf_dir, "train.txt"), 'r') as fp:
    train_info = [entry.strip().split(',') for entry in fp.readlines()]
image_dir = os.path.join(psuf_dir, 'images')
train_data_psuf = get_data_from_info(image_dir, None, train_info)"""
# Split data into train, validate and test
train_split, test_data_hackathon = train_test_split(_train_data_hackathon, test_size=0.2, shuffle=True, random_state=42)
train_data_hackathon, valid_data_hackathon = train_test_split(train_split, test_size=0.2, shuffle=True, random_state=43)

#balance_training_data(train_data_hackathon, seed=72)
#balance_training_data(valid_data_hackathon, seed=73)
#balance_training_data(test_data_hackathon, seed=74)

Splitting large images...
original data len: 355
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0001.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0022.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0011.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0016.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0008.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0015.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0021.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0009.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0020.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0012.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0014.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0002.nii.gz
split image: /home/jupyter/RIS/data/HACKATHON/images/train/0019.nii.gz
split image: /home/jupyter/R

## Setup transforms

In [18]:
# Crop foreground
crop_foreground = CropForegroundd(
    keys=["image"],
    source_key="image",
    margin=(5, 5, 0),
    select_fn = lambda x: x != 0
)
# Crop Z
crop_z = RelativeCropZd(keys=["image"], relative_z_roi=(0.05, 0.15))
# Window width and level (window center)
WW, WL = 1500, -600
ct_window = CTWindowd(keys=["image"], width=WW, level=WL)
# Random flip axis
rand_x_flip = RandFlipd(keys=["image"], spatial_axis=0, prob=0.50)
rand_y_flip = RandFlipd(keys=["image"], spatial_axis=1, prob=0.50)
rand_z_flip = RandFlipd(keys=["image"], spatial_axis=2, prob=0.50)
# Rand affine transform
rand_affine = RandAffined(
    keys=["image"],
    prob=0.50,
    rotate_range=(0, 0, np.pi/12),
    shear_range=(0.07, 0.07, 0.0),
    translate_range=(0, 0, 0),
    scale_range=(0.07, 0.07, 0.0),
    padding_mode="zeros"
)
# Pad image to have hight at least 30
spatial_pad = SpatialPadd(keys=["image"], spatial_size=(-1, -1, 30))
# Resize image x and y
resize_fator = 0.5
resize = Resized(keys=["image"], spatial_size=(int(512*resize_fator), int(512*resize_fator), -1), mode="trilinear")
# Apply Gaussian noise
rand_gaussian_noise = RandGaussianNoised(keys=["image"], prob=0.25, mean=0.0, std=0.1)

#### Create transforms

In [19]:
common_transform = Compose([
    LoadImaged(keys=["image"]),
    ct_window,
    CTSegmentation(keys=["image"]),
    AddChanneld(keys=["image"]),
    resize,
    crop_foreground,
    crop_z,
    spatial_pad,
])
hackathon_train_transform = Compose([
    common_transform,
    rand_x_flip,
    rand_y_flip,
    rand_z_flip,
    rand_affine,
    rand_gaussian_noise,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_valid_transfrom = Compose([
    common_transform,
    #rand_x_flip,
    #rand_y_flip,
    #rand_z_flip,
    #rand_affine,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_test_transfrom = Compose([
    common_transform,
    ToTensord(keys=["image"]),
]).flatten()
psuf_transforms = Compose([
    LoadImaged(keys=["image"]),
    AddChanneld(keys=["image"]),
    ToTensord(keys=["image"]),
])

## Setup data

In [20]:
#set_determinism(seed=100)
train_dataset = PersistentDataset(data=train_data_hackathon[:], transform=hackathon_train_transform, cache_dir=dirs["persistent"])
valid_dataset = PersistentDataset(data=valid_data_hackathon[:], transform=hackathon_valid_transfrom, cache_dir=dirs["persistent"])
test_dataset = PersistentDataset(data=test_data_hackathon[:], transform=hackathon_test_transfrom, cache_dir=dirs["persistent"])
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    #shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    sampler=ImbalancedDatasetSampler(train_data_hackathon, callback_get_label=lambda x, i: x[i]['_label']),
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    #shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    sampler=ImbalancedDatasetSampler(valid_data_hackathon, callback_get_label=lambda x, i: x[i]['_label']),
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)
test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    #shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)

## Setup network, loss function, optimizer and scheduler

In [25]:
network = nets.DenseNet169(spatial_dims=3, in_channels=1, out_channels=1).to(device)
# pos_weight for class imbalance
pos_weight = 1#calculate_class_imbalance(train_data_hackathon)
pos_weight = torch.Tensor([pos_weight]).to(device)
loss_function = torch.nn.BCEWithLogitsLoss(pos_weight)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-5, weight_decay=0)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95, last_epoch=-1)

## Setup validator and trainer

In [26]:
# Setup validator and trainer
valid_post_transforms = Compose([
    Activationsd(keys="pred", sigmoid=True),
    #Activationsd(keys="pred", softmax=True),
])
validator = Validator(
    device=device,
    val_data_loader=valid_loader,
    network=network,
    post_transform=valid_post_transforms,
    amp=using_gpu,
    non_blocking=using_gpu
)

trainer = Trainer(
    device=device,
    out_dir=dirs["out"],
    out_name="DenseNet169",
    max_epochs=120,
    validation_epoch = 1,
    validation_interval = 1,
    train_data_loader=train_loader,
    network=network,
    optimizer=optimizer,
    loss_function=loss_function,
    lr_scheduler=None,
    validator=validator,
    amp=using_gpu,
    non_blocking=using_gpu
)

## Run trainer

In [27]:
train_output = trainer.run()

Training started: 14/04/2021 05:38:51
INFO:ignite.engine.engine.Trainer:Engine run resuming from iteration 0, epoch 0 until 120 epochs
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 1/64 -- loss: 0.6746 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 2/64 -- loss: 0.6139 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 3/64 -- loss: 0.5903 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 4/64 -- loss: 0.6171 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 5/64 -- loss: 0.6355 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 6/64 -- loss: 0.6136 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 7/64 -- loss: 0.5627 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 8/64 -- loss: 0.5954 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 9/64 -- loss: 0.8035 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 10/64 -- loss: 0.7971 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 11/64 -- loss: 0.6679 
INFO:ignite.engine.engine.T

KeyboardInterrupt: 

## Setup tester

In [ ]:
tester = Tester(
    device=device,
    test_data_loader=test_loader,
    load_dir=train_output,
    out_dir=dirs["out"],
    network=network,
    post_transform=valid_post_transforms,
    non_blocking=using_gpu,
    amp=using_gpu
)

## Run tester

In [ ]:
tester.run()